# HCAN 361 Final Project: Spring 2021

## Access to Obstetric Care in Hospitals: Comparing Alabama, California, and New York

In [1]:
USE [HCAN361_FinalProj]

--select item no that includes OBGYN/WOMEN care 
--this is the info we will have to consider when looking 
--at what classifies ob care 
SELECT *
FROM [dbo].[AS2019_DataDictionary]
WHERE [item No#] is not null 
and [Field Description] like '%Obstetric%'
or [Field Description] like '%women%'
or [Field Description] like '%gyn%'
order by [Item No#]

(16 rows affected)

Total execution time: 00:00:00.065

Item No#,FieldName,Estimated,Length,Start,End,Field Description,Survey Question,Access Table,Notes
201,OBLEV,N,1,2296,2296,Obstetric unit care level,C.3.,as19svc1,1=provides services for uncomplicated maternity and newborn cases; 2=provides service for all uncomplicated and most complicated cases; 3=provides services for all serious illnesses and abnormalities
202,OBBD,N,4,2297,2300,Obstetric care beds,C.3.,as19svc1,NULL
229,OBHOS,N,1,2675,2675,Obstetrics care - hospital,C.3.,as19svc1,"1=yes, 0=no"
230,OBSYS,N,1,2676,2676,Obstetrics care - health system,C.3.,as19svc1,"1=yes, 0=no"
231,OBVEN,N,1,2677,2677,Obstetrics care - joint venture,C.3.,as19svc1,"1=yes, 0=no"
735,WOMHCHOS,N,1,3193,3193,Women’s health center/services - hospital,C.104.,as19svc3,"1=yes, 0=no"
736,WOMHCSYS,N,1,3194,3194,Women’s health center/services - health system,C.104.,as19svc3,"1=yes, 0=no"
737,WOMHCVEN,N,1,3195,3195,Women’s health center/services - joint venture,C.104.,as19svc3,"1=yes, 0=no"
1029,TEOB,N,8,4828,4835,Obstetrics/gynecology - total employed,E.12.b.1.,as19util2,Number of physicians with privileges
1037,TCOB,N,8,4892,4899,Obstetrics/gynecology - total individual contract,E.12.b.2.,as19util2,Number of physicians with privileges


In [2]:
--Step 1: Select ID of OBGYN-Obstetric care Hospital Item No# 229
select ID
into #key_ID
from [dbo].AS19SVC1
where OBHOS=1;

select top 10 *
from #key_ID

(2219 rows affected)

(10 rows affected)

Total execution time: 00:00:00.058

ID
6040005
6110003
6110008
6110050
6110090
6110120
6110167
6110200
6110240
6110280


In [3]:
--Step 2:Find target population of interest for these services
--women age 15 to 54 and over (B01001e30-->B01001e40) and put into temp table #pop
select GEOID_Data,
	 B01001e30+B01001e31+B01001e32+B01001e33+B01001e34
	+B01001e35+B01001e36+B01001e37+B01001e38+B01001e39+B01001e40
	 AS pop15plus
into #pop
from [dbo].[Census_ACS5Yr_2018]
order by GEOID_Data;

select top 10 *
from #pop

(74001 rows affected)

(10 rows affected)

Total execution time: 00:00:00.148

GEOID_Data,pop15plus
14000US01097001001,530.00000000
14000US01097001002,541.00000000
14000US01097001100,458.00000000
14000US01097001200,746.00000000
14000US01097001302,597.00000000
14000US01097001400,342.00000000
14000US01097001501,428.00000000
14000US01097001502,418.00000000
14000US01097001800,790.00000000
14000US01097001901,548.00000000


In [4]:
--Step 3: Ranking and Sorting ADI into integrers for our table 
select [GEOID_Data],[State_Name],[HRSA_RURAL],
case when [Median_ADI_NatRank]<=15 then 2
     when ([Median_ADI_NatRank]>15 and [Median_ADI_NatRank]<=75) then 1
	 when [Median_ADI_NatRank]>85 then 0
	 else 99 end as [ADI]
into #characteristic
from [dbo].[US_Census_Tract_Charac]

select top 10 *
from #characteristic

(74001 rows affected)

(10 rows affected)

Total execution time: 00:00:00.129

GEOID_Data,State_Name,HRSA_RURAL,ADI
14000US06037462002,California,0,1
14000US06037462100,California,0,1
14000US06037462201,California,0,1
14000US06037462202,California,0,1
14000US06037462301,California,0,1
14000US06037462302,California,0,2
14000US06037462400,California,0,2
14000US06037462500,California,0,2
14000US06037462600,California,0,2
14000US06037462700,California,0,2


In [5]:
--Step 4: Build Matrix (hospital travel time: having access is a hospital within 30 min of travel time)
select GEOID_Data, count(distinct Provider_ID) as hospitals_win_30min, min([Total_TravelTime]) 
as closest_ob_hospital
into #key_table
from
(
select A.*, B.pop15plus
FROM [HCAN361_FinalProj].[dbo].[Travel_Matrix] as A
left join #pop as B on A.GEOID_Data=B.GEOID_Data
where Provider_ID in (select ID from #key_ID)
and Total_TravelTime<=30
) as A
group by GEOID_Data
order by GEOID_Data;

select top 10 *
from #key_table

(63611 rows affected)

(10 rows affected)

Total execution time: 00:00:03.914

GEOID_Data,hospitals_win_30min,closest_ob_hospital
14000US17031836100,12,5.148786175924159
14000US12057013804,1,24.866727259596342
14000US06059063805,9,9.198925848038602
14000US13139001606,3,18.69219221655334
14000US12086003921,3,11.904926749992256
14000US48201253100,2,9.558551959177986
14000US24033803610,6,9.35275904056845
14000US06067009403,5,18.997288770500415
14000US06107001901,1,6.424790490756051
14000US53061053509,1,20.303316236027765


In [6]:
--Step 5: Final Table which joins our key table with our characteristic table 
select  a.GEOID_Data
      , c.State_Name
	  , c.ADI
	  , c.HRSA_RURAL
	  , a.pop15plus
      , isnull(b.hospitals_win_30min,0) as  hospitals_win_30min
	  , isnull(b.closest_ob_hospital,361) as closest_ob_hospital 
into [HCAN361_25].[dbo].[finalMap_Group4]
from #pop as a
left join #key_table as b on A.GEOID_Data=B.GEOID_Data
left join #characteristic as c on A.GEOID_Data=C.GEOID_Data;

alter table [HCAN361_25].[dbo].[finalMap_Group4]
alter column GEOID_Data varchar(30) not null;

alter table [HCAN361_25].[dbo].[finalMap_Group4]
add constraint pk_Grp4_map primary key (GEOID_Data);

select top 10 *
from [HCAN361_25].[dbo].[finalMap_Group4]

(74001 rows affected)

(10 rows affected)

Total execution time: 00:00:02.209

GEOID_Data,State_Name,ADI,HRSA_RURAL,pop15plus,hospitals_win_30min,closest_ob_hospital
14000US01001020100,Alabama,1,0,512.00000000,1,25.957991285000737
14000US01001020200,Alabama,1,0,520.00000000,1,25.405170626841795
14000US01001020300,Alabama,1,0,928.00000000,2,21.55145796874138
14000US01001020400,Alabama,1,0,878.00000000,2,19.68035044606142
14000US01001020500,Alabama,1,0,2637.00000000,2,17.671510736752616
14000US01001020600,Alabama,1,0,1030.00000000,2,20.94884137264777
14000US01001020700,Alabama,1,0,1268.00000000,2,18.26627435869834
14000US01001020801,Alabama,1,0,679.00000000,1,26.427979775217487
14000US01001020802,Alabama,1,0,3372.00000000,1,24.944714472723007
14000US01001020900,Alabama,1,0,1624.00000000,0,361


In [7]:
--Step 6: Build lat/long point level hospital location table
SELECT ID, cast(LAT as float) as LAT, cast(LONG as float) as LONG, MNAME
INTO [HCAN361_25].[dbo].[provider_Loc_Group4]
FROM dbo.AS19DEM
WHERE ID IN (SELECT ID FROM [dbo].AS19SVC1 where OBHOS='1');

alter table [HCAN361_25].[dbo].[provider_Loc_Group4]
alter column ID varchar(30) not null;

alter table [HCAN361_25].[dbo].[provider_Loc_Group4]
add constraint pk_Grp4_provLoc primary key (ID);

select top 10 *
from [HCAN361_25].[dbo].[provider_Loc_Group4]

(2219 rows affected)

(10 rows affected)

Total execution time: 00:00:00.253

ID,LAT,LONG,MNAME
6040005,18.4553,-67.1319,Hospital Buen Samaritano
6110003,43.906,-69.8904,Mid Coast Hospital
6110008,43.6514,-70.2649,Northern Light Mercy Hospital
6110050,44.8074,-68.7526,Northern Light Eastern Maine Medical Center
6110090,44.385,-68.2022,Mount Desert Island Hospital
6110120,44.4122,-68.9939,Waldo County General Hospital
6110167,44.0467,-70.7141,Bridgton Hospital
6110200,46.8797,-68.0083,Cary Medical Center
6110240,45.1877,-69.2379,Northern Light Mayo Hospital
6110280,44.6265,-70.163,Franklin Memorial Hospital


In [8]:
--Step 7: Create Tables  
--Table 1: Median Number of Facilities by Attributes
select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hospitals_win_30min)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then hospitals_win_30min end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then hospitals_win_30min end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then hospitals_win_30min end) OVER () AS [High Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then hospitals_win_30min end) OVER () AS [Middle Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then hospitals_win_30min end) OVER () AS [Low Class]
from [HCAN361_21].[dbo].[finalMap]

--below code concatenates
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hospitals_win_30min)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then hospitals_win_30min end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then hospitals_win_30min end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then hospitals_win_30min end) OVER (PARTITION BY State_Name) AS [High Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then hospitals_win_30min end) OVER (PARTITION BY State_Name) AS [Middle Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then hospitals_win_30min end) OVER (PARTITION BY State_Name) AS [Low Class]
from [HCAN361_21].[dbo].[finalMap]
where State_Name in ('Alabama','New York','California')

) as A
order by State_Name;

(4 rows affected)

Total execution time: 00:00:03.443

State_Name,Statewide,Urban,Rural,High Class,Middle Class,Low Class
Alabama,1,2,0,3,1,1
California,6,6,0,6,6,7
New York,6,7,1,9,3,11
United States,3,4,1,6,3,3


In [ ]:
--Step 8: Create Tables  
--Table 2: Median Travel Time to Closest Facility by Attributes
select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY closest_ob_hospital)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then closest_ob_hospital end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then closest_ob_hospital end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then closest_ob_hospital end) OVER () AS [High Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then closest_ob_hospital end) OVER () AS [Middle Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then closest_ob_hospital end) OVER () AS [Low Class]
from [HCAN361_21].[dbo].[finalMap]

--below code concatenates
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY closest_ob_hospital)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then closest_ob_hospital end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then closest_ob_hospital end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then closest_ob_hospital end) OVER (PARTITION BY State_Name) AS [High Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then closest_ob_hospital end) OVER (PARTITION BY State_Name) AS [Middle Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then closest_ob_hospital end) OVER (PARTITION BY State_Name) AS [Low Class]
from [HCAN361_21].[dbo].[finalMap]
where State_Name in ('Alabama','New York','California')

) as A
order by State_Name;

(4 rows affected)

Total execution time: 00:00:03.718

State_Name,Statewide,Urban,Rural,High Class,Middle Class,Low Class
Alabama,18.7361998433624,14.786833410931235,361,12.045710895580434,18.23679001827328,13.246745839328934
California,10.637444469867773,10.272895188291322,361,10.564982038956664,10.718867517900343,8.62388496698154
New York,10.934336788049656,10.407003822880869,24.857320437051918,10.282233472270603,13.269632973845264,7.067581392814598
United States,12.825890441898753,11.657927876395092,26.407661602629716,11.344064671908352,13.453609743744188,10.350400744253776
